In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse, parse_qs
import datetime

#### 1. 뉴스 속보 페이지에서 정치/경제/사회 기사 링크 수집 
링크 : https://news.naver.com/main/ranking/popularDay.nhn

In [2]:
# 수집해야할 날짜의 범위
def news_link_crawler(date_range, base_url, params):

    ranking_news_list = []
    
    for date in date_range:
        params['date'] = date
#         print('date', date)

        resp = requests.get(base_url,params=params)
        soup = BeautifulSoup(resp.text,'lxml')
        ranking_wrap = soup.select('div.ranking_section li dt a')

        date_news_dict = {}
        news_list = []

        # 정치, 경제, 사회로 범위 한정
        for ranking in ranking_wrap[:-15]:
            news_list.append(urljoin(base_url,ranking.get('href')))

        date_news_dict[date] = news_list

        ranking_news_list.append(date_news_dict)
    
    return ranking_news_list


#### 뉴스 기사에서 기사제목/ 섹션 / 요약봇의 기사요약 내용 / 이미지 추출

In [3]:
def news_body_crawler(ranking_news_list):
    finally_news_list = []

    for ranking_list in ranking_news_list:
        for yyyymmdd, ranking in ranking_list.items():
            for ranking_news in ranking:

                news_dict = {}
                
                resp_news = requests.get(ranking_news)
                soup_news = BeautifulSoup(resp_news.text)

                # 기사 제목
                header_wrap = soup_news.select('div.article_info h3')
                title = header_wrap[0].text

                # 섹션
                section = soup_news.find('em', class_='guide_categorization_item')

                # 기사 ID
                parsed_url = urlparse(ranking_news)
                qs = parse_qs(parsed_url.query)
                oid = qs['oid'][0]
                aid = qs['aid'][0]
#                 print(oid, aid)

                # 내용
                summary_url = 'https://tts.news.naver.com/article/{}/{}/summary'.format(oid, aid)
                resp_summary = requests.get(summary_url)
                soup_summary = BeautifulSoup(resp_summary.text)
                summary = soup_summary.text.split(':')[-1].replace('}','').replace("\\","")

                news_dict = {
                    '날짜' : yyyymmdd[:4]+'-'+yyyymmdd[4:6]+'-'+yyyymmdd[6:],
                    'title' : title,
                    'category' : section.text,
                    'summary' : summary.replace('"','')
                }

                # 이미지 
                image_wrap = soup_news.find('span', class_="end_photo_org")

                if image_wrap:
                    news_dict['img'] = image_wrap.find('img').get('src')
                finally_news_list.append(news_dict)
                
    return finally_news_list

In [4]:
def save_df_to_csv(finally_news_list):
    df = pd.DataFrame(finally_news_list)
    df.to_csv('finally_naver_news_crawling.csv', encoding='cp949', index=False)

In [5]:
date_range = (pd.date_range(start='20200501', end=datetime.date.today())).strftime("%Y%m%d").tolist()
base_url= 'https://news.naver.com/main/ranking/popularDay.nhn'
params = {
    'rankingType' : 'popular_day',
    'date' : ''
}

NewsList = news_link_crawler(date_range, base_url, params)
# NewsList[0]

FinallyNewsList = news_body_crawler(NewsList)
# len(finally_news_list),finally_news_list[0]

save_df_to_csv(FinallyNewsList)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
